# **Shopee Code League - Order Brushing**

This notebook is for Shopee Code League Competition, Order Brushing, to detecting abnormal user behaviour. The competition details can be obtained from [here](https://www.kaggle.com/c/order-brushing-shopee-code-league/overview).

Team name: CY Team (Malaysia)

Team member:
1. Chong Li Chuin
2. Ee Yeo Keat

# **Import libraries and read dataset**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from datetime import datetime
from google.colab import files

In [2]:
data = pd.read_csv('order_brush_order.csv').sort_values(by=['event_time','shopid','userid'])
data

,orderid,shopid,userid,event_time
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05
...,...,...,...,...
114113,31507183252446,149254894,193333760,2019-12-31 23:59:43
206585,31507187390691,147941492,40258063,2019-12-31 23:59:47
160657,31507191066627,154074176,2338306,2019-12-31 23:59:51
126212,31507191066628,187123853,2338306,2019-12-31 23:59:51


In [4]:
date = data['event_time']
date

150060    2019-12-27 00:00:00
81020     2019-12-27 00:00:00
25985     2019-12-27 00:00:00
146598    2019-12-27 00:00:02
80355     2019-12-27 00:00:05
                 ...         
114113    2019-12-31 23:59:43
206585    2019-12-31 23:59:47
160657    2019-12-31 23:59:51
126212    2019-12-31 23:59:51
179881    2019-12-31 23:59:56
Name: event_time, Length: 222750, dtype: object

In [29]:
d = []
t = []

for i in date[:100]:
  date_time = datetime.strptime(i,"%Y-%m-%d %H:%M:%S")
  d = date_time.date()
  t = date_time.time()
  print(d)
  print(t)

2019-12-27
00:00:00
2019-12-27
00:00:00
2019-12-27
00:00:00
2019-12-27
00:00:02
2019-12-27
00:00:05
2019-12-27
00:00:05
2019-12-27
00:00:05
2019-12-27
00:00:08
2019-12-27
00:00:08
2019-12-27
00:00:08
2019-12-27
00:00:08
2019-12-27
00:00:08
2019-12-27
00:00:08
2019-12-27
00:00:11
2019-12-27
00:00:11
2019-12-27
00:00:11
2019-12-27
00:00:12
2019-12-27
00:00:16
2019-12-27
00:00:16
2019-12-27
00:00:16
2019-12-27
00:00:16
2019-12-27
00:00:18
2019-12-27
00:00:23
2019-12-27
00:00:23
2019-12-27
00:00:23
2019-12-27
00:00:24
2019-12-27
00:00:24
2019-12-27
00:00:24
2019-12-27
00:00:24
2019-12-27
00:00:24
2019-12-27
00:00:24
2019-12-27
00:00:24
2019-12-27
00:00:24
2019-12-27
00:00:24
2019-12-27
00:00:26
2019-12-27
00:00:26
2019-12-27
00:00:29
2019-12-27
00:00:31
2019-12-27
00:00:32
2019-12-27
00:00:33
2019-12-27
00:00:35
2019-12-27
00:00:36
2019-12-27
00:00:37
2019-12-27
00:00:37
2019-12-27
00:00:40
2019-12-27
00:00:42
2019-12-27
00:00:43
2019-12-27
00:00:43
2019-12-27
00:00:44
2019-12-27
00:00:44


In [21]:
d = []
t = []

for i in date[:10]:
  date_time = datetime.strptime(i,"%Y-%m-%d %H:%M:%S")
  d,t = date_time.apply(lambda x:x.time()), date_time.apply(lambda x:x.date())
  print(d)
  print(t)

AttributeError: ignored

In [9]:
data['Time'],data['Date']= date.apply(lambda x:x.time()), date.apply(lambda x:x.date())
data

AttributeError: ignored

In [ ]:
def check_concentrate_rate():
  concentrate_rate = #Number of Orders within 1 hour / Number of Unique Buyers within 1 hour
  if concentrate_rate >= 3:
    shop_list.append(shopid)
  
  return shop_list

In [ ]:
save_data = data.shopid.unique
save_data

<bound method Series.unique of 150060      6042309
81020       8715449
25985     104804492
146598    190969466
80355       2859407
            ...    
114113    149254894
206585    147941492
160657    154074176
126212    187123853
179881     31844790
Name: shopid, Length: 222750, dtype: int64>

In [ ]:
df.to_csv('CY_Team_submission4.csv') 
files.download('CY_Team_submission4.csv')